# Скачивание данных и библиотек

In [1]:
!pip install -U pip setuptools wheel
!pip install transformers
!pip install deep-translator
!pip install wget
!pip install fitbert

import wget
import os
import zipfile

if not os.path.isfile('test_pipeline_questions.csv'):
  wget.download("https://storage.yandexcloud.net/vk-cup-data/test_pipeline_questions.csv")

if not os.path.isfile('2021 VK Cup ML Finals.zip'):
  wget.download("https://www.dropbox.com/sh/ds8wh42kftccd6f/AAAOtEW26zOauBc7KOx-FY4ba?dl=1")
  with zipfile.ZipFile('2021 VK Cup ML Finals.zip', 'r') as zip_ref:
    zip_ref.extractall()

     |████████████████████████████████| 1.6 MB 6.4 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     |████████████████████████████████| 2.6 MB 8.2 MB/s 
     |████████████████████████████████| 895 kB 70.5 MB/s 
     |████████████████████████████████| 3.3 MB 61.5 MB/s 
     |████████████████████████████████| 636 kB 75.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=d58e60e01a80fa5a575e26596341080438e185ecbff6ac8e3cc8d752af299a8f
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
     |████████████████████████████████| 216 kB 8.2 MB/s 
     |████████████████████████████████| 44 kB 4.2 MB/s 
     |███████████████████████████

In [ ]:
 wget.download("https://disk.yandex.ru/d/dpzZI40DILgr_Q")

'dpzZI40DILgr_Q'

In [2]:
import pandas as pd
from google.colab import files
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from tqdm.notebook import tqdm
tqdm.pandas()
GLOBAL_DEVICE = 'cuda'


# Код общего назначения

In [6]:
class Data:
  def __init__(self, test_size=0.25):
    self.train_mode = None
    self.train = None
    self.test = None
    self.test_size = test_size
  
  def __get_rus(self, seed):
    rus_data = pd.read_csv("rus_train_dataset.csv", delimiter='|')
    rus_data = rus_data.rename(columns={'Unnamed: 0': 'index'})
    if self.test_size != 1:
      rus_train, rus_test = train_test_split(rus_data, random_state=seed, test_size=self.test_size)
    else:
      rus_test = rus_data
      rus_train = rus_data.head(0)
    return rus_train, rus_test

  def __get_eng(self):
    eng_data = pd.read_csv("en_to_rus_train_dataset.csv", delimiter='|')
    eng_data = eng_data.rename(columns={'Unnamed: 0': 'index'}) 
    return eng_data

  def load_train_test(self, mode, seed):
    print(f'Данные загружаютсяб режим {mode}')
    rus_train, rus_test = self.__get_rus(seed)
    if mode == "RUS":
      self.train = rus_train
      self.test = rus_test

    elif mode == "ALL":
      eng_train = self.__get_eng()
      self.train = rus_train.append(eng_train, ignore_index=True)
      self.test = rus_test
    else:
      ValueError('Wrong value for mode')

      self.train['question'] = self.train['question'].replace('ё', 'е')
      self.test['question'] = self.test['question'].replace('ё', 'е')
    return None

  def get_train(self, mode='RUS', seed=42):
    if self.train_mode != mode:
      self.load_train_test(mode, seed)
    return self.train

  def get_test(self, mode='RUS', seed=42):
    if self.train_mode != mode:
      self.load_train_test(mode, seed)
    return self.test
  
  @staticmethod
  def stack_columns(data):
    data = data.set_index('index')
    data_stacked = data[['ps_0', 'ps_1',	'ps_2']].stack()
    data_stacked = data_stacked.reset_index()
    data_stacked['level_1'] = data_stacked['level_1'].str[-1]
    data_stacked = data_stacked.rename(columns={'level_1': 'answer_num',	0: 'answer'})
    data_merged = data.merge(data_stacked, on='index')
    data_merged['target'] = data_merged['right_answer_id'].astype(int) == data_merged['answer_num'].astype(int)
    out = data_merged[['index', 'question', 'answer', 'target']]
    return out

  def get_train_for_model(mode='ALL', seed=42):
    train_inner = self.get_train(mode, seed)
    test = self.get_test(mode, seed)
    return Data.stack_columns(train_inner), Data.stack_columns(test)

from transformers import BertTokenizer, BertModel
import torch

def get_test_accuracy(data, model):
  test = data.get_test()
  my_prediction = model.predict(data)
  merged = test.merge(my_prediction, on='index')
  accuracy = (merged['right_answer_id'] == merged['my_prediction']).sum() / merged.shape[0]
  return merged, accuracy

In [ ]:
question_words = ['Кто',
'Кого',
'Кому',
'Кем',
'Что',
'Ком',
'Какой',
'Какого',
'Какому',
'Каким',
'Каком',
'Какую',
'Какая',
'Какой',
'Который',
'Которого',
'Которому',
'Которым',
'Где',
'Когда',
'Почему',
'Зачем',
'Куда',
'Откуда',
'Сколько',
'Чей',
'Как',
'чем',
'Какое',
'Какие',
'Скольким',
'Скольких',
'Столько',
'Стольких',
'Именно',
'чем',
'чего',
'насколько',
'каких',
'какова',
'Самый',
'Самая',
'Самое',
'Какими'
]
question_words_lower = set([word.lower() for word in question_words])

test_data['not_found_words'] = test_data['question'].apply(lambda x: len(set(x.lower().split(' ')) & question_words_lower)) == 0

test_data[test_data['not_found_words']]

In [ ]:
for word in question_words_lower:
  test_data['question'] = test_data['question'].str.lower().str.replace(word, 'exchnage')


# Baseline

In [ ]:
class Baseline:
  def __init__(self):
    model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
    self.tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
    self.model = GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()
  
  def calc_likelihood(self, text):
    tokens_ids = self.tokenizer.encode(text, return_tensors="pt").cuda()
    prob = self.model(tokens_ids)

    tokens = tokens_ids[0].cpu().tolist()
    prob = prob[0].cpu()[0]

    likelyhood = sum([prob[i, j].item() for i, j in enumerate(tokens)])
    return likelyhood

  def predict_one(self, question, answers):
    scores = [self.calc_likelihood(question + " " +  answer) for answer in answers]
    return np.argmax(scores)

  def predict(self, data):
    predict_data = data.get_test()
    prediction = predict_data.progress_apply(lambda x: self.predict_one(x['question'], [x['ps_0'], x['ps_1'], x['ps_2']]), axis=1)
    return pd.DataFrame({ 'index': predict_data['index'],
                          'my_prediction': prediction})



In [11]:
class Baseline:
  def __init__(self):
    model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
    self.tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
    self.model = GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()
  
  def calc_likelihood(self, text):
    tokens_ids = self.tokenizer.encode(text, return_tensors="pt").cuda()
    prob = self.model(tokens_ids)

    tokens = tokens_ids[0].cpu().tolist()
    prob = prob[0].cpu()[0]

    likelyhood = prob[len(tokens) - 1, tokens[-1]].item()
    return likelyhood

  def predict_one(self, question, answers):
    scores = [self.calc_likelihood(question + " " +  answer) for answer in answers]
    return np.argmax(scores)

  def predict(self, data):
    predict_data = data.get_test()
    prediction = predict_data.progress_apply(lambda x: self.predict_one(x['question'], [x['ps_0'], x['ps_1'], x['ps_2']]), axis=1)
    return pd.DataFrame({ 'index': predict_data['index'],
                          'my_prediction': prediction})

data = Data(test_size=1)
model = Baseline()

merged, accuracy = get_test_accuracy(data, model)
print(accuracy)

Данные загружаютсяб режим RUS
Данные загружаютсяб режим RUS


  0%|          | 0/4061 [00:00<?, ?it/s]

0.3220881556266929


# Baseilen + word position

In [17]:
class BaselineWordPos:
  question_words = ['Кто',
                    'Кого',
                    'Кому',
                    'Кем',
                    'Что',
                    'Ком',
                    'Какой',
                    'Какого',
                    'Какому',
                    'Каким',
                    'Каком',
                    'Какую',
                    'Какая',
                    'Какой',
                    'Который',
                    'Которого',
                    'Которому',
                    'Которым',
                    'Где',
                    'Когда',
                    'Почему',
                    'Зачем',
                    'Куда',
                    'Откуда',
                    'Сколько',
                    'Чей',
                    'Как',
                    'чем',
                    'Какое',
                    'Какие',
                    'Скольким',
                    'Скольких',
                    'Столько',
                    'Стольких',
                    'Именно',
                    'чем',
                    'чего',
                    'насколько',
                    'каких',
                    'какова',
                    'Самый',
                    'Самая',
                    'Самое',
                    'Какими'
                    ]


  def __init__(self):
    model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
    self.tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
    self.model = GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()
  
  def calc_likelihood(self, text):

    tokens_ids = self.tokenizer.encode(text, return_tensors="pt").cuda()
    prob = self.model(tokens_ids)

    tokens = tokens_ids[0].cpu().tolist()
    prob = prob[0].cpu()[0]


    likelyhood = sum([prob[i, j].item() for i, j in enumerate(tokens)])
    return likelyhood


  def get_changed_question(self, question, answer):
    changed_question = question
    word_found = True

    for word in self.question_words:
      changed_question = " ".join([answer if word_from_question.lower() == word.lower() else word_from_question for word_from_question in changed_question.split(' ')])

    if changed_question == question:
      changed_question = question + ' ' + answer
    return changed_question

  def predict_one(self, question, answers):
    question = question.replace('?', '')
    scores = [self.calc_likelihood(self.get_changed_question(question, answer.replace('.', '')), ) for answer in answers]
    return np.argmax(scores)

  def predict(self, data):
    predict_data = data.get_test()
    prediction = predict_data.progress_apply(lambda x: self.predict_one(x['question'], [x['ps_0'], x['ps_1'], x['ps_2']]), axis=1)
    return pd.DataFrame({ 'index': predict_data['index'],
                          'my_prediction': prediction})

data = Data(test_size=0.1)
model = BaselineWordPos()

merged, accuracy = get_test_accuracy(data, model)
accuracy

Данные загружаютсяб режим RUS
Данные загружаютсяб режим RUS


  0%|          | 0/407 [00:00<?, ?it/s]

Выходные данные были обрезаны до нескольких последних строк (5000).
        [-7.4961,  9.8819, -7.1173,  ..., -0.1187, -0.7642, -2.8173],
        ...,
        [-6.5122,  8.2295, -6.1528,  ...,  0.6415, -1.5100, -2.1332],
        [-7.1941, 12.6865, -7.5967,  ..., -2.0733, -1.6649, -2.5132],
        [-7.7831, 13.7504, -8.0974,  ...,  0.2643, -1.5763, -4.2320]],
       grad_fn=<SelectBackward>)
tensor([[-6.7146,  4.6748, -6.8461,  ..., -3.3125, -3.5799, -1.0824],
        [-6.1457,  8.3241, -7.2069,  ..., -1.7379,  0.2108, -2.0633],
        [-6.3014,  6.0374, -6.5182,  ..., -2.8032, -1.4673, -1.4763],
        ...,
        [-6.5424,  8.6871, -6.2344,  ..., -0.4051, -1.6619, -2.1126],
        [-7.2429, 12.7930, -7.7345,  ..., -2.7930, -2.1120, -2.3107],
        [-7.8992, 14.1429, -8.2851,  ..., -0.6116, -1.5477, -3.6155]],
       grad_fn=<SelectBackward>)
tensor([[-7.0657,  4.8321, -6.7419,  ..., -3.4255, -0.8421, -1.5380],
        [-7.6011,  9.3939, -8.5370,  ..., -2.4161, -2.2996, -0.6703]

KeyboardInterrupt: ignored

# Baseilen + word position + translation

In [7]:
from fitbert import FitBert
from transformers import RobertaForMaskedLM, RobertaTokenizer, AutoTokenizer, AutoModel
from deep_translator import GoogleTranslator

class BaselineWordPosTranslate:
  question_words = ['Who',
                  'What',
                  'Which',
                  'Whose',
                  'Whom',
                  'Where',
                  'When',
                  'How',
                  'Why']

  def __init__(self):
    model_name_or_path = "gpt2-large"
    self.tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
    self.model = GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()
    self.translator = GoogleTranslator(source='ru', target='en')
  
  def calc_likelihood(self, text):
    tokens_ids = self.tokenizer.encode(text, return_tensors="pt").cuda()
    prob = self.model(tokens_ids)

    tokens = tokens_ids[0].cpu().tolist()
    prob = prob[0].cpu()[0]

    likelyhood = sum([prob[i, j].item() for i, j in enumerate(tokens)])
    return likelyhood


  def get_changed_question(self, question, answer):
    
    changed_question = question
    word_found = True

    for word in self.question_words:
      print(word.lower())
      changed_question = " ".join([answer if word_from_question.lower() == word.lower() else word_from_question for word_from_question in changed_question.split(' ')])

    if changed_question == question:
      changed_question = question + ' ' + answer
    

    return changed_question

  def predict_one(self, question, answers):
    question = self.translator.translate(question)
    answers = [answer if answer.isdigit() else self.translator.translate(answer) for answer in answers]

    scores = [self.calc_likelihood(self.get_changed_question(question, answer)) for answer in answers]
    return np.argmax(scores)

  def predict(self, data):
    predict_data = data.get_test()
    prediction = predict_data.progress_apply(lambda x: self.predict_one(x['question'], [x['ps_0'], x['ps_1'], x['ps_2']]), axis=1)
    return pd.DataFrame({ 'index': predict_data['index'],
                          'my_prediction': prediction})


data = Data(test_size=0.1)
model = BaselineWordPosTranslate()

merged, accuracy = get_test_accuracy(data, model)

Данные загружаютсяб режим RUS
Данные загружаютсяб режим RUS


  0%|          | 0/407 [00:00<?, ?it/s]

In [9]:
accuracy

0.3464373464373464

# Baseline with pos and not

In [ ]:
class BaselineWordPosNot:
  question_words = ['Кто',
                    'Кого',
                    'Кому',
                    'Кем',
                    'Что',
                    'Ком',
                    'Какой',
                    'Какого',
                    'Какому',
                    'Каким',
                    'Каком',
                    'Какую',
                    'Какая',
                    'Какой',
                    'Который',
                    'Которого',
                    'Которому',
                    'Которым',
                    'Где',
                    'Когда',
                    'Почему',
                    'Зачем',
                    'Куда',
                    'Откуда',
                    'Сколько',
                    'Чей',
                    'Как',
                    'чем',
                    'Какое',
                    'Какие',
                    'Скольким',
                    'Скольких',
                    'Столько',
                    'Стольких',
                    'Именно',
                    'чем',
                    'чего',
                    'насколько',
                    'каких',
                    'какова',
                    'Самый',
                    'Самая',
                    'Самое',
                    'Какими'
                    ]
    
  negative_words = ['не',
                    'нет']

  def __init__(self):
    model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
    self.tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
    self.model = GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()
  
  def calc_likelihood(self, text):
    tokens_ids = self.tokenizer.encode(text, return_tensors="pt").cuda()
    prob = self.model(tokens_ids)

    tokens = tokens_ids[0].cpu().tolist()
    prob = prob[0].cpu()[0]

    likelyhood = sum([prob[i, j].item() for i, j in enumerate(tokens)])
    return likelyhood


  def get_changed_question(self, question, answer):

    answer = answer.replace('.', '')
    answer = answer.replace(',', '')
    changed_question = question
    word_found = True
    for word in self.question_words:
      changed_question = " ".join([answer if word_from_question.lower() == word.lower() else word_from_question for word_from_question in changed_question.split(' ')])

    if changed_question == question:
      changed_question = question + ' ' + answer

    changed_question = changed_question.replace('?', '')
    return changed_question

  def is_negative(self, question):
    changed_question = question
    for word in self.negative_words:
      changed_question = " ".join(['' if word_from_question.lower() == word.lower() else word_from_question for word_from_question in changed_question.split(' ')])
    return changed_question, changed_question != question

  def predict_one(self, question, answers):
    positive_question, flag_is_negative = self.is_negative(question)
  
    scores = [self.calc_likelihood(self.get_changed_question(positive_question, answer)) for answer in answers]
    if flag_is_negative:
      out = np.argmin(scores)
    else:
      out = np.argmax(scores)

    return out

  def predict(self, data):
    predict_data = data.get_test()
    prediction = predict_data.progress_apply(lambda x: self.predict_one(x['question'], [x['ps_0'], x['ps_1'], x['ps_2']]), axis=1)
    return pd.DataFrame({ 'index': predict_data['index'],
                          'my_prediction': prediction})



# Delete common words

In [ ]:
class BaselineWordPosNot:
  question_words = ['Кто',
                    'Кого',
                    'Кому',
                    'Кем',
                    'Что',
                    'Ком',
                    'Какой',
                    'Какого',
                    'Какому',
                    'Каким',
                    'Каком',
                    'Какую',
                    'Какая',
                    'Какой',
                    'Который',
                    'Которого',
                    'Которому',
                    'Которым',
                    'Где',
                    'Когда',
                    'Почему',
                    'Зачем',
                    'Куда',
                    'Откуда',
                    'Сколько',
                    'Чей',
                    'Как',
                    'чем',
                    'Какое',
                    'Какие',
                    'Скольким',
                    'Скольких',
                    'Столько',
                    'Стольких',
                    'Именно',
                    'чем',
                    'чего',
                    'насколько',
                    'каких',
                    'какова',
                    'Самый',
                    'Самая',
                    'Самое',
                    'Какими'
                    ]
    
  negative_words = ['не',
                    'нет']

  def __init__(self):
    model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
    self.tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
    self.model = GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()
  
  def calc_likelihood(self, text):
    tokens_ids = self.tokenizer.encode(text, return_tensors="pt").cuda()
    prob = self.model(tokens_ids)

    tokens = tokens_ids[0].cpu().tolist()
    prob = prob[0].cpu()[0]

    likelyhood = sum([prob[i, j].item() for i, j in enumerate(tokens)])
    return likelyhood


  def get_changed_question(self, question, answer):

    answer = answer.replace('.', '')
    answer = answer.replace(',', '')
    changed_question = question
    word_found = True
    for word in self.question_words:
      changed_question = " ".join([answer if word_from_question.lower() == word.lower() else word_from_question for word_from_question in changed_question.split(' ')])

    if changed_question == question:
      changed_question = question + ' ' + answer

    changed_question = changed_question.replace('?', '')
    return changed_question

  def is_negative(self, question):
    changed_question = question
    for word in self.negative_words:
      changed_question = " ".join(['' if word_from_question.lower() == word.lower() else word_from_question for word_from_question in changed_question.split(' ')])
    return changed_question, changed_question != question

  def predict_one(self, question, answers):
    positive_question, flag_is_negative = self.is_negative(question)
  
    scores = [self.calc_likelihood(self.get_changed_question(positive_question, answer)) for answer in answers]
    if flag_is_negative:
      out = np.argmin(scores)
    else:
      out = np.argmax(scores)

    return out

  def predict(self, data):
    predict_data = data.get_test()
    prediction = predict_data.progress_apply(lambda x: self.predict_one(x['question'], [x['ps_0'], x['ps_1'], x['ps_2']]), axis=1)
    return pd.DataFrame({ 'index': predict_data['index'],
                          'my_prediction': prediction})

# Try BERTS ML

In [ ]:
d = Data()
d.get_test()

Данные загружаютсяб режим RUS


,index,question,ps_0,ps_1,ps_2,right_answer_id
3286,3286,Какой баскетболист получил Оскар в 2018 году?,Майкл Джордан,ЛеБрон Джеймс,Коби Брайант,2
2463,2463,На флаге какого региона России до сих пор сохр...,Красноярский край,Тульская область,Владимирская область,2
4058,4058,В каком году была выпущена приставка «Playstat...,2000,2001,1999,0
746,746,Какой премии НЕ был удостоен Андрей Гейм?,Шнобелевская премия,Нобелевская премия,Грэмми,2
1979,1979,Какую страну переименовали в 2018 году?,Свазиленд,Никарагуа,Панама,0
...,...,...,...,...,...,...
2609,2609,Какая из этих черепах НЕ является хищником?,Красноухая черепаха,Грифовая черепаха,Среднеазиатская черепаха,2
1454,1454,Кто запатентовал первый вариант телефона?,Александр Белл,Филипп Райс,Никола Тесла,0
3717,3717,Где родился Евгений Онегин?,Петербург,Москва,Болдино,0
1849,1849,Кто основал город Омск?,Пётр I,Иван Бухгольц,Иван Грозный,1


In [ ]:
# Мастерская

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

import random
import math
import time


class EmbedderNet(nn.Module):
    def __init__(self, hid_dim, n_hidden_layers, embedding_dim=):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedder = Embedder()

        self.input_layer = nn.Linear(in_features=embedding_dim, out_features=output_shape)        
        self.hidden_layers = [nn.Linear(in_features=hid_dim, out_features=hid_dim) for i in range(n_hidden_layers)]
        self.output_layer = [nn.Linear(in_features=hid_dim, out_features=hid_dim) for i in range(n_hidden_layers)] 

        # self.dropout = nn.Dropout(p=0.1)
        
    def forward(self, data):
      question = self.embedder.sentence_embedding(data['question'])
      answer = self.embedder.sentence_embedding(data['answer'])

      input = torch.cat(question, answer)
      
      layer_output = self.input_layer(input)
      
      for linnear_layer in self.hidden_layers:
        layer_output = linnear_layer(layer_output)
      
      result = self.output_layer(layer_output)
      result = 

      
      output = 
        return 

def train(model, iterator, optimizer, scheduler, criterion, clip, train_history=None, valid_history=None):
    model.train()
    
    epoch_loss = 0
    history = []
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        # Let's clip the gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
 
        epoch_loss += loss.item()
        
        history.append(loss.cpu().data.numpy())
        if (i+1)%10==0:
            fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))

            clear_output(True)
            ax[0].plot(history, label='train loss')
            ax[0].set_xlabel('Batch')
            ax[0].set_title('Train loss')
            if train_history is not None:
                ax[1].plot(train_history, label='general train history')
                ax[1].set_xlabel('Epoch')
            if valid_history is not None:
                ax[1].plot(valid_history, label='general valid history')
            plt.legend()
            
            plt.show()

    if len(valid_history):
        scheduler.step(valid_history[-1])
        
    return epoch_loss / len(iterator)
  
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    history = []
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0)

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs







PAD_IDX = TRG.vocab.stoi['<pad>']


optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=2)
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

train_history = []
valid_history = []

N_EPOCHS = 3
CLIP = 1

best_valid_loss = float('inf')

import torch, gc

gc.collect()
torch.cuda.empty_cache()

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = GLOBAL_DEVICE,
    sort_key=_len_sort_key
)

for epoch in range(N_EPOCHS):
    torch.cuda.empty_cache()
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, scheduler, criterion, CLIP, train_history, valid_history)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    train_history.append(train_loss)
    valid_history.append(valid_loss)
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

,index,question,answer,target
0,4056,"Как называются рецепторы в организме человека,...",Хеморецепторы,False
1,4056,"Как называются рецепторы в организме человека,...",Барорецепторы,True
2,4056,"Как называются рецепторы в организме человека,...",Механорецепторы,False
3,3007,Какая женщина дважды была лауреатом Нобелевско...,Софья Васильевна Ковалевская,False
4,3007,Какая женщина дважды была лауреатом Нобелевско...,Мария Склодовская-Кюри,True
...,...,...,...,...
6085,3507,Какой из этих газов имеет запах?,Пропан,False
6086,3507,Какой из этих газов имеет запах?,Угарный газ,False
6087,3174,Вид из какого города изображен на банкноте в 1...,Красноярск,False
6088,3174,Вид из какого города изображен на банкноте в 1...,Новгород,False


# Косинусные расстояния

In [8]:
import sklearn

In [13]:
import torch
class Embedder:
    DEVICE = 'cuda'
    def __init__(self, bert_name="DeepPavlov/rubert-base-cased-sentence"):
        self.tokenizer = BertTokenizer.from_pretrained(bert_name)
        self.model = BertModel.from_pretrained(bert_name, output_attentions = True, output_hidden_states = True).to(self.DEVICE)
        # self.model.cuda()

    def sentence_embedding(self, text):
        embeddings = []
 
        ts = self.tokenizer.encode_plus(text)
        th_inputs = torch.tensor([ts['input_ids']]).to(self.DEVICE)
        th_type = torch.tensor([ts['token_type_ids']]).to(self.DEVICE)
        with torch.no_grad():
            outputs = self.model(th_inputs, token_type_ids=th_type)
        return outputs.last_hidden_state[0][0].cpu().numpy().reshape(-1, 1)



def get_test_accuracy(data, model):
  test = data.get_test()
  my_prediction = model.predict(data)
  merged = test.merge(my_prediction, on='index')
  accuracy = (merged['right_answer_id'] == merged['my_prediction']).sum() / merged.shape[0]
  return merged, accuracy

class BertCosine:
  def __init__(self):
    self.embeder = Embedder()


  def predict_one(self, question, answers):
    question_embed = self.embeder.sentence_embedding(question)
    scores = [sklearn.metrics.pairwise.cosine_similarity(question_embed, self.embeder.sentence_embedding(answer)) for answer in answers]
    return np.argmax(scores)

  def predict(self, data):
    predict_data = data.get_test()
    prediction = predict_data.progress_apply(lambda x: self.predict_one(x['question'], [x['ps_0'], x['ps_1'], x['ps_2']]), axis=1)
    return pd.DataFrame({ 'index': predict_data['index'],
                          'my_prediction': prediction})

data = Data(test_size=1)
model = BertCosine()

merged, accuracy = get_test_accuracy(data, model)
print(accuracy)

Данные загружаютсяб режим RUS
Данные загружаютсяб режим RUS


  0%|          | 0/4061 [00:00<?, ?it/s]

0.32602807190347205


In [ ]:
# Косинусные расстояния + перевод

# Сетка на растоянии

In [ ]:
class 

In [ ]:
question = embedder.sentence_embedding("Как называется половина основного времени матча в футболе?")
answer1 = embedder.sentence_embedding("Тайм")
answer2 = embedder.sentence_embedding("Гейм")
answer3 = embedder.sentence_embedding("Период")


In [ ]:
def cosine_distance(x1, x2=None, eps=1e-8):
    x2 = x1 if x2 is None else x2
    w1 = x1.norm(p=2, dim=1, keepdim=True)
    w2 = w1 if x2 is x1 else x2.norm(p=2, dim=1, keepdim=True)
    return 1 - torch.mm(x1, x2.t()) / (w1 * w2.t()).clamp(min=eps)

print(cosine_distance(question, answer1))
print(cosine_distance(question, answer2))
print(cosine_distance(question, answer3))


tensor([[0.4487]], device='cuda:0')
tensor([[0.5241]], device='cuda:0')
tensor([[0.5469]], device='cuda:0')


In [ ]:
class EmbedderNet:
  def __init__(self):
    model_name_or_path = "rubert-base-cased-sentence"
    self.tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
    self.model = GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()
  
  def calc_likelihood(self, text):
    tokens_ids = self.tokenizer.encode(text, return_tensors="pt").cuda()
    prob = self.model(tokens_ids)

    tokens = tokens_ids[0].cpu().tolist()
    prob = prob[0].cpu()[0]

    likelyhood = sum([prob[i, j].item() for i, j in enumerate(tokens)])
    return likelyhood

  def predict_one(self, question, answers):
    scores = [self.calc_likelihood(question + " " +  answer) for answer in answers]
    return np.argmax(scores)

  def predict(self, data):
    predict_data = data.get_test()
    prediction = predict_data.progress_apply(lambda x: self.predict_one(x['question'], [x['ps_0'], x['ps_1'], x['ps_2']]), axis=1)
    return pd.DataFrame({ 'index': predict_data['index'],
                          'my_prediction': prediction})

# Перевод + fitBert

In [ ]:
from fitbert import FitBert
from transformers import RobertaForMaskedLM, RobertaTokenizer, AutoTokenizer, AutoModel

class FitBertMy:
  question_words = ['Кто',
                    'Кого',
                    'Кому',
                    'Кем',
                    'Что',
                    'Ком',
                    'Какой',
                    'Какого',
                    'Какому',
                    'Каким',
                    'Каком',
                    'Какую',
                    'Какая',
                    'Какой',
                    'Который',
                    'Которого',
                    'Которому',
                    'Которым',
                    'Где',
                    'Когда',
                    'Почему',
                    'Зачем',
                    'Куда',
                    'Откуда',
                    'Сколько',
                    'Чей',
                    'Как',
                    'чем',
                    'Какое',
                    'Какие',
                    'Скольким',
                    'Скольких',
                    'Столько',
                    'Стольких',
                    'Именно',
                    'чем',
                    'чего',
                    'насколько',
                    'каких',
                    'какова',
                    'Самый',
                    'Самая',
                    'Самое',
                    'Какими'
                    ]


  def __init__(self, model_name="DeepPavlov/rubert-base-cased-sentence", mask_token='<mask>'):
  
    self.model_name = model_name
    self.mask_token = mask_token
    self.model_input=RobertaForMaskedLM.from_pretrained(model_name)
    self.tokenizer_input=RobertaTokenizer.from_pretrained(model_name)
    self.fb = FitBert(model=self.model_input, tokenizer=self.tokenizer_input, mask_token=self.mask_token)


  def get_changed_question(self, question):
    changed_question = question
    word_found = True
    for word in self.question_words:
      changed_question = " ".join([self.mask_token if word_from_question.lower() == word.lower() else word_from_question for word_from_question in changed_question.split(' ')])
      if changed_question != question:
        break

    if changed_question == question:
      changed_question = question + ' ' + self.mask_token
    return changed_question

  def predict_one(self, question, answers):

    changed_question = self.get_changed_question(question)
    ranked_options = self.fb.rank(changed_question, options=answers)

    my_answer = ranked_options[0]
    # print(my_answer)
    # print(answers)
    return answers.index(my_answer)

  def predict(self, data):
    predict_data = data.get_test()
    prediction = predict_data.progress_apply(lambda x: self.predict_one(x['question'], [x['ps_0'], x['ps_1'], x['ps_2']]), axis=1)
    return pd.DataFrame({ 'index': predict_data['index'],
                          'my_prediction': prediction})


# filBert + перевод

In [6]:
from fitbert import FitBert
from transformers import RobertaForMaskedLM, RobertaTokenizer, AutoTokenizer, AutoModel
from deep_translator import GoogleTranslator


class TranslateFitBertMy:


  question_words =['Who'
                  'What'
                  'Which'
                  'Whose'
                  'Whom'
                  'Where'
                  'When'
                  'How'
                  'Why']

  def __init__(self, model_name="roberta-large", mask_token='<mask>'):
  
    self.model_name = model_name
    self.mask_token = mask_token
    self.translator = GoogleTranslator(source='ru', target='en')
    self.model_input=RobertaForMaskedLM.from_pretrained(model_name)
    self.tokenizer_input=RobertaTokenizer.from_pretrained(model_name)
    self.fb = FitBert(model_name=self.model_name, model=self.model_input, tokenizer=self.tokenizer_input, mask_token=self.mask_token)


  def get_changed_question(self, question):
    changed_question = question
    word_found = True
    for word in self.question_words:
      changed_question = " ".join([self.mask_token if word_from_question.lower() == word.lower() else word_from_question for word_from_question in changed_question.split(' ')])
      if changed_question != question:
        break

    if changed_question == question:
      changed_question = question + ' ' + self.mask_token
    return changed_question

  def predict_one(self, question, answers):
    if not question.isdigit():
      question = self.translator.translate(question)
    question = question.replace("?", '.')
  
    answers = [answer.replace(".", '') if answer.isdigit() else self.translator.translate(answer.replace(".", '')) for answer in answers]
    changed_question = self.get_changed_question(question)
    ranked_options = self.fb.rank(changed_question, options=answers)

    my_answer = ranked_options[0]
    # print(my_answer)
    # print(answers)
    return answers.index(my_answer)

  def predict(self, data):
    predict_data = data.get_test()
    prediction = predict_data.progress_apply(lambda x: self.predict_one(x['question'], [x['ps_0'], x['ps_1'], x['ps_2']]), axis=1)
    return pd.DataFrame({ 'index': predict_data['index'],
                          'my_prediction': prediction})

data = Data(test_size=0.1)
model = TranslateFitBertMy()

merged, accuracy = get_test_accuracy(data, model)
print(accuracy)


Downloading:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

device: cuda
using custom model: ['RobertaForMaskedLM']
Данные загружаютсяб режим RUS
Данные загружаютсяб режим RUS


  0%|          | 0/407 [00:00<?, ?it/s]

0.3538083538083538


In [12]:
from fitbert import FitBert
from transformers import RobertaForMaskedLM, RobertaTokenizer, BertForMaskedLM, AutoModelForQuestionAnswering, BertTokenizer, BertTokenizer, BertModel, XLMRobertaModel, XLMRobertaTokenizer, BertForQuestionAnswering, DistilBertForMaskedLM, DistilBertTokenizer

# in theory you can pass a model_name and tokenizer, but currently only
# bert-large-uncased and BertTokenizer are available
# this takes a while and loads a whole big BERT into memory
model_name = "DeepPavlov/rubert-base-cased-sentence"

model_input=BertForMaskedLM.from_pretrained(model_name)

tokenizer_input=BertTokenizer.from_pretrained(model_name)

fb = FitBert(model=model_input, tokenizer=tokenizer_input, mask_token='***mask***')

# Where was Eugene Onegin born? Petersburg Moscow Boldino 
masked_string = "***mask*** was Eugene Onegin born"
options = ['Petersburg', 'Moscow', 'Boldino']
 
ranked_options = fb.rank(masked_string, options=options, with_prob=True)
ranked_options

NameError: ignored

In [62]:
from fitbert import FitBert
from transformers import RobertaForMaskedLM, RobertaTokenizer, BertForMaskedLM, AutoModelForQuestionAnswering, BertTokenizer, BertTokenizer, BertModel, XLMRobertaModel, XLMRobertaTokenizer, BertForQuestionAnswering, DistilBertForMaskedLM, DistilBertTokenizer

# in theory you can pass a model_name and tokenizer, but currently only
# bert-large-uncased and BertTokenizer are available
# this takes a while and loads a whole big BERT into memory
model_name = "roberta-large"

model_input=RobertaForMaskedLM.from_pretrained(model_name)

tokenizer_input=RobertaTokenizer.from_pretrained(model_name)

fb = FitBert(model=model_input, tokenizer=tokenizer_input, mask_token='***mask***')

# Where was Eugene Onegin born? Petersburg Moscow Boldino 
masked_string = "***mask*** was Eugene Onegin born"
options = ['Petersburg', 'Moscow', 'Boldino']
 
ranked_options = fb.rank(masked_string, options=options, with_prob=True)
ranked_options

device: cuda
using custom model: ['RobertaForMaskedLM']


(['Boldino', 'Petersburg', 'Moscow'],
 [5.3558937906927895e-06, 8.577973176215892e-07, 6.463145751922639e-08])

In [ ]:
# rus_data['question_trans'] = rus_data['question'].progress_apply(translator.translate)

In [ ]:
# rus_data['question_trans'] = rus_data['question'].progress_apply(translator.translate)
# rus_data['ps_0_trans'] = rus_data['ps_0'].astype(str).progress_apply(lambda x: x if x.isdigit() else translator.translate(x))
# rus_data['ps_1_trans'] = rus_data['ps_1'].astype(str).progress_apply(lambda x: x if x.isdigit() else translator.translate(x))
# rus_data['ps_2_trans'] = rus_data['ps_2'].astype(str).progress_apply(lambda x: x if x.isdigit() else translator.translate(x))

In [ ]:
# question_words_eng =['Who'
#                 'What'
#                 'Which'
#                 'Whose'
#                 'Whom'
#                 'Where'
#                 'When'
#                 'How'
#                 'Why']


In [ ]:
!pip install fitbert

     |████████████████████████████████| 216 kB 5.4 MB/s 
     |████████████████████████████████| 44 kB 2.8 MB/s 
     |████████████████████████████████| 81 kB 9.3 MB/s 
  Created wheel for fitbert: filename=fitbert-0.9.0-py3-none-any.whl size=214165 sha256=84a14cb6c4854d13afe0494005942725dad93bd14811865a93ea5a535ecc23d0
  Stored in directory: /root/.cache/pip/wheels/6c/33/95/cba855a6db54d40ce15dc0e8ab7b83e82b4c43501681ae10d8
  Created wheel for dill: filename=dill-0.2.5-py3-none-any.whl size=52439 sha256=4e21e27d605a65a147a85fde4aa5f18985a7ecf7df2b40a89c01283686131913
  Stored in directory: /root/.cache/pip/wheels/56/3a/d6/508ddf62a7a83b0b851169e9871b47c882f0f78f22874ed6ab
Successfully built fitbert dill
  Attempting uninstall: dill
    Found existing installation: dill 0.3.4
    Uninstalling dill-0.3.4:
      Successfully uninstalled dill-0.3.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the

device: cuda
using custom model: ['RobertaForMaskedLM']


(['4', '6', '5'],
 [0.0007613090565428138, 0.000325696513755247, 0.00024577759904786944])

In [ ]:
test_data = d.get_test()
test_data.iloc[8, :]

Данные загружаютсяб режим RUS


index                                                              8
question           Сколько фильмов в серии «Пираты Карибского моря»?
ps_0                                                               6
ps_1                                                               5
ps_2                                                               4
right_answer_id                                                    1
Name: 8, dtype: object

In [ ]:
class FitBertTrans:
  question_words = ['Кто',
                    'Кого',
                    'Кому',
                    'Кем',
                    'Что',
                    'Ком',
                    'Какой',
                    'Какого',
                    'Какому',
                    'Каким',
                    'Каком',
                    'Какую',
                    'Какая',
                    'Какой',
                    'Который',
                    'Которого',
                    'Которому',
                    'Которым',
                    'Где',
                    'Когда',
                    'Почему',
                    'Зачем',
                    'Куда',
                    'Откуда',
                    'Сколько',
                    'Чей',
                    'Как',
                    'чем',
                    'Какое',
                    'Какие',
                    'Скольким',
                    'Скольких',
                    'Столько',
                    'Стольких',
                    'Именно',
                    'чем',
                    'чего',
                    'насколько',
                    'каких',
                    'какова',
                    'Самый',
                    'Самая',
                    'Самое',
                    'Какими'
                    ]
    
  negative_words = ['не',
                    'нет']

  def __init__(self):
    model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
    self.tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
    self.model = GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()
  
  def calc_likelihood(self, text):
    tokens_ids = self.tokenizer.encode(text, return_tensors="pt").cuda()
    prob = self.model(tokens_ids)

    tokens = tokens_ids[0].cpu().tolist()
    prob = prob[0].cpu()[0]

    likelyhood = sum([prob[i, j].item() for i, j in enumerate(tokens)])
    return likelyhood


  def get_changed_question(self, question, answer):

    answer = answer.replace('.', '')
    answer = answer.replace(',', '')
    changed_question = question
    word_found = True
    for word in self.question_words:
      changed_question = " ".join([answer if word_from_question.lower() == word.lower() else word_from_question for word_from_question in changed_question.split(' ')])

    if changed_question == question:
      changed_question = question + ' ' + answer

    changed_question = changed_question.replace('?', '')
    return changed_question

  def is_negative(self, question):
    changed_question = question
    for word in self.negative_words:
      changed_question = " ".join(['' if word_from_question.lower() == word.lower() else word_from_question for word_from_question in changed_question.split(' ')])
    return changed_question, changed_question != question

  def predict_one(self, question, answers):
    positive_question, flag_is_negative = self.is_negative(question)
  
    scores = [self.calc_likelihood(self.get_changed_question(positive_question, answer)) for answer in answers]
    if flag_is_negative:
      out = np.argmin(scores)
    else:
      out = np.argmax(scores)

    return out

  def predict(self, data):
    predict_data = data.get_test()
    prediction = predict_data.progress_apply(lambda x: self.predict_one(x['question'], [x['ps_0'], x['ps_1'], x['ps_2']]), axis=1)
    return pd.DataFrame({ 'index': predict_data['index'],
                          'my_prediction': prediction})

NameError: ignored

In [ ]:
rus_data['question_trans']

In [ ]:
rus_data

In [ ]:
rus_data['question_trans'] = rus_data['question'].progress_apply(translator.translate)

In [ ]:
rus_data.to_pickle('rus_data_question')

In [ ]:
rus_data.iloc[0, 1]

# Проверка моделей

In [ ]:
def save_file(data, name):
  data.to_excel(name)
  files.download(name)
  return None

In [ ]:
data = Data(test_size=1)
model = Baseline()

merged, accuracy = get_test_accuracy(data, model)
save_file(merged, 'Baseline_res.xlsx')
print(accuracy)

  0%|          | 0/4061 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

0.35508495444471805


In [8]:
data = Data(test_size=1)
model = BertCosine()

merged, accuracy = get_test_accuracy(data, model)
print(accuracy)

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/711M [00:00<?, ?B/s]

Данные загружаютсяб режим RUS
Данные загружаютсяб режим RUS


  0%|          | 0/407 [00:00<?, ?it/s]

0.3783783783783784


In [ ]:
data = Data(test_size=1)
model = BaselineWordPos()

merged, accuracy = get_test_accuracy(data, model)
save_file(merged, 'BaselineWordPos.xlsx')
print(accuracy)

Данные загружаютсяб режим RUS
Данные загружаютсяб режим RUS


  0%|          | 0/4061 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

0.36050233932528936


In [ ]:
data = Data(test_size=0.1)
model = BaselineWordPos()

merged, accuracy = get_test_accuracy(data, model)
print(accuracy)

Данные загружаютсяб режим RUS
Данные загружаютсяб режим RUS


  0%|          | 0/407 [00:00<?, ?it/s]

0.3488943488943489


In [ ]:
data = Data(test_size=1)
model = BaselineWordPosNot()

merged, accuracy = get_test_accuracy(data, model)
print(accuracy)


Данные загружаютсяб режим RUS
Данные загружаютсяб режим RUS


  0%|          | 0/4061 [00:00<?, ?it/s]

0.3548387096774194


In [ ]:
print('1')

1
